In [30]:
# Includes básicos

import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
from utils.funciones import *

In [31]:
# Primer dataset de entrenamiento

# y_train es una lista con dos columnas, id y estado de la bomba
y_train_filename="datos/y_train_data.csv"
df_ytrain=pd.read_csv(y_train_filename)


# x_train es una lista con los datos de cada bomba, sin el estado.
x_train_filename="datos/x_train_data.csv"
df_xtrain=pd.read_csv(x_train_filename)


# segundo dataset que se usará en el proceso de machine learning
x_test_filename="datos/x_test_data.csv"
df_xtest=pd.read_csv(x_test_filename)
df_xtest.set_index('id')
df_xtest.sort_index(inplace=True)


# Fusionamos nuestras dos primeras bases de datos
df_train=df_xtrain.merge(df_ytrain,how='inner',on='id',sort=True)
df_train.set_index('id')
df_train.sort_index(inplace=True)

# Comprobamos valores duplicados y llenamos NAN
df_train.drop_duplicates(inplace=True)
df_xtest.drop_duplicates(inplace=True)

df_train=fill_object_nan(df_train)
df_xtest=fill_object_nan(df_xtest)

# Introducimos columnas de información para el date recorded.
def add_date_columns(df):
    df["date_recorded"]=pd.to_datetime(df['date_recorded'],format='%Y-%m-%d',errors='coerce')
    df["date_recorded_year"]=df["date_recorded"].dt.year
    df["date_recorded_month"]=df["date_recorded"].dt.month

    # Establecemos 1999 como año de cosntrucción de aquellos bombas que no se conocen.
    df.construction_year=np.where(df.construction_year>0,df.construction_year,1999)
    df.population=np.where(df.population>0,df.population,\
        round(np.median(df.population.loc[df.population>0].mean()),0))
    return df

df_train=add_date_columns(df_train)
df_xtest=add_date_columns(df_xtest)

for col in tipos_col(df_train)["obj"]:
    # simplificamos las variables categóricas, limitándolas a los diez valores por variable con presencia de > 2%.
    df_train[col] = agrupa_categoricas(df_train,col,10,0)
    
    # conservamos mismos tipos escogidos en xtest
    if col in df_xtest.columns:
        valores_categoricas=df_train[col].unique()
        df_xtest[col] = df_xtest[col].apply(lambda x: x if x in valores_categoricas else 'unknown')
    
# Simplificamos el target para obtener 2 en primicia
df_train['status_group_simpl']=df_train['status_group']
df_train['status_group_simpl']=np.where(df_train['status_group_simpl']=='functional','functional','non functional')

#Substutimos valores extra por "desconocidos"
df_train.replace("other","unknown",inplace=True)
df_xtest.replace("other","unknown",inplace=True)

In [32]:
" Existen muchos valores con longitud y latitud cero. sin embargo, tienen una región asignada. Por ello, vamos a asignarles valores al azar entre los puntos de cada región"

df_train_local=df_train.loc[df_train["longitude"]!=0]
df_xtest_local=df_xtest.loc[df_xtest["longitude"]!=0]

regiones_maximas=df_train_local[["gps_height","longitude","latitude","region_code"]].groupby(by=["region_code"]).max().add_suffix("_max")
regiones_minimas=df_train_local[["gps_height","longitude","latitude","region_code"]].groupby(by=["region_code"]).min().add_suffix("_min")

regiones=regiones_maximas.join(regiones_minimas)

#inputamos valores de colocación basados en región en una nueva variable (df_train_one)
df_train_zero=df_train.loc[df_train["longitude"]==0]
df_xtest_zero=df_xtest.loc[df_xtest["longitude"]==0]

var_imputables=["gps_height","longitude","latitude"]

for region in regiones.index:
    for variable in var_imputables:

        min=regiones.loc[[region]][variable+"_min"].values[0]
        max=regiones.loc[[region]][variable+"_max"].values[0]

        tam_train=df_train_zero[variable].loc[df_train_zero["region_code"]==region].shape
        tam_xtest=df_xtest_zero[variable].loc[df_xtest_zero["region_code"]==region].shape

        valores_train=np.random.random_sample(size=tam_train)*(max-min)+np.ones(tam_train)*min
        valores_xtest=np.random.random_sample(size=tam_xtest)*(max-min)+np.ones(tam_xtest)*min

        df_train_zero[variable].loc[df_train_zero["region_code"]==region]=valores_train
        df_xtest_zero[variable].loc[df_xtest_zero["region_code"]==region]=valores_xtest

df_train_loc_imput=df_train_zero[["region_code"]+var_imputables]
df_xtest_loc_imput=df_xtest_zero[["region_code"]+var_imputables]

df_train_one=df_train.copy()
df_xtest_one=df_xtest.copy()

df_train_one.loc[df_train_zero.index, var_imputables]=df_train_zero[var_imputables]
df_xtest_one.loc[df_xtest_zero.index, var_imputables]=df_xtest_zero[var_imputables]

C:\Users\Rucufuto\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\Rucufuto\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\core\indexing.py:1965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj._check_is_chained_assignment_possible()


In [33]:
# Añadimos quién es el pozo más cercano. 
# Para esto nos basaremos UNICAMENTE en los pozos de la misma región del set de entrenamiento. 
# Esto se dará incluso si estamos en el set de test.
# Es un proceso largo, puede durar más de una hora. Por eso, miraré si he calculado ya el punto.

import joblib
from pathlib import Path

ruta='datos/'

# Miramos si ya he trabajado estos datos antes. Si no, los creamos. Esta operación puede llevar hasta una hora.
archivo_gps_train='df_train_one_gps.pkl'
path= Path(ruta+archivo_gps_train)
if path.is_file():
    print("archivo de datos train gps encontrado")
    print(f"leyendo de {ruta+archivo_gps_train}")
    df_train_one_gps=joblib.load(ruta+archivo_gps_train)
    df_train_one_gps=df_train_one_gps.set_index('id')
    df_train_one=df_train_one.join(df_train_one_gps['dist_mas_cercano'],how='inner')
else:
    df_train_one["dist_mas_cercano"]=lugar_mas_cercano(df_train_one[["longitude","latitude","region_code"]],"gps",True)
    joblib.dump(df_train_one, ruta+archivo_gps_train)

# Repetimos el paso con los datos del dataset objetivo.
archivo_gps_xtest='df_xtest_one_gps.pkl'
path= Path(ruta+archivo_gps_xtest)
if path.is_file():
    print("archivo de datos xtest gps encontrado")
    print(f"leyendo de {ruta+archivo_gps_xtest}")
    df_xtest_one_gps=joblib.load(ruta+archivo_gps_xtest)
    df_xtest_one=df_xtest_one.join(df_xtest_one_gps['dist_mas_cercano'],how='inner')
else:
    df_xtest_one["dist_mas_cercano"]=lugar_mas_cercano_xtest(df_train_one[["longitude","latitude","region_code"]],\
        df_xtest_one[["longitude","latitude","region_code"]],"gps",True)
    joblib.dump(df_xtest_one, ruta+archivo_gps_xtest)

archivo de datos train gps encontrado
leyendo de datos/df_train_one_gps.pkl
archivo de datos xtest gps encontrado
leyendo de datos/df_xtest_one_gps.pkl


In [34]:
#Guardamos variables, guardamos indices como indice.

df_train_one.set_index('id',inplace=True)
df_xtest_one.set_index('id',inplace=True)

In [35]:


# Convertimos o serializamos las clases en formato pickle pkl
joblib.dump(df_train_one, "datos/df_train_datos.pkl")
joblib.dump(df_xtest_one, "datos/df_xtest_datos.pkl")
joblib.dump(regiones,'datos/regiones.pkl')
print(" --- Pickle labelEncoder dump executed ---")

 --- Pickle labelEncoder dump executed ---


PARTE 2

In [36]:
ruta='datos/'
archivo_xtrain='df_train_datos.pkl'
archivo_xtest='df_xtest_datos.pkl'
df_train_one=joblib.load(ruta+archivo_xtrain)
df_xtest_one=joblib.load(ruta+archivo_xtest)

# Categorías a trabajar
lista_numericas=['amount_tsh', 'construction_year', 'longitude', 'latitude', 'gps_height','population','dist_mas_cercano']
lista_categoricas=["waterpoint_type","source_type","quality_group","quantity_group","payment_type","extraction_type_class","permit","scheme_name","funder","installer"]
listas_targets=['status_group','status_group_simpl']

lista_categoricas=["quantity_group","extraction_type_class","waterpoint_type"]
listas=[lista_numericas,lista_categoricas,listas_targets]
listas=sum(listas,[])
listas_test=sum([lista_numericas,lista_categoricas],[])

df_train_two=df_train_one[listas]
df_xtest_two=df_xtest_one[listas_test]




In [37]:
# Aplicamos dummies

print("aplicamos dummies a df_train")
df_train_two_dm=aplica_dummies(df_train_two,lista_categoricas)
# primero conseguido

aplicamos dummies a df_train
aplicamos dummies
columna: quantity_group
columna: extraction_type_class
columna: waterpoint_type


In [38]:
print("aplicamos dummies a df_xtest")
df_xtest_three=aplica_dummies(df_xtest_two,lista_categoricas)
print("ambos conseguidos")

aplicamos dummies a df_xtest
aplicamos dummies
columna: quantity_group
columna: extraction_type_class
columna: waterpoint_type
ambos conseguidos


In [39]:
# guardamos en nueva df

df_train_two_dm['target_trio']=df_train_two_dm['status_group'].replace(['functional', 'functional needs repair','non functional'],[0, 1,2])
df_train_two_dm['target_duo']=df_train_two_dm['status_group_simpl'].replace(['functional', 'functional needs repair','non functional'],[0, 1,1])
df_train_three=df_train_two_dm.drop(columns=['status_group','status_group_simpl'])

In [40]:
# si hay columnas extra de dummmies en df_train_two que no hay en df_xtest, las añadimos ahora, con valor cero en todas ellas.

col_comunes=set(df_train_three.columns).symmetric_difference(set(df_xtest_three.columns)).symmetric_difference(set(['target_duo','target_trio']))
for col in col_comunes:
    df_xtest_three[col] = 0

col_sin_target=list(set(df_train_three.columns).intersection(set(df_xtest_three.columns)))
col_sin_target.sort()
col_con_target=col_sin_target[:]
col_con_target.extend(['target_duo','target_trio'])

df_train_three=df_train_three[col_con_target]
df_xtest_three=df_xtest_three[col_sin_target]

In [41]:
# Convertimos o serializamos las clases en formato pickle pkl

joblib.dump(df_train_three, "datos/df_train_datos2.pkl")
joblib.dump(df_xtest_three, "datos/df_xtest_datos2.pkl")
print(" --- Pickle labelEncoder dump executed ---")

 --- Pickle labelEncoder dump executed ---
